In [3]:
import os 
import sys
import re
import time
import csv
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import ik_parsing as ik
import data_extraction as de
import helper_functions as hp
import defs
from datetime import datetime
from collections import defaultdict
import itertools
import zipfile
import gzip
import json
import concurrent.futures
from multiprocessing import Pool
import matplotlib.pyplot as plt
import math

head, tail = os.path.split(os.getcwd())

In [34]:
head, tail = os.path.split(os.getcwd())
# list_of_filenames = ['1000294.txt']
list_of_filenames = os.listdir(os.path.join(head, 'sample_cases_10k'))
print(len(list_of_filenames))
list_of_banks = pd.read_csv('../bank_names.csv', header=None)
list_of_banks = list_of_banks[0].tolist()
list_of_banks = ik.preprocess_company_names(list_of_banks, bank=True)
list_of_banks[:5]

9923


['axis bank', 'bandhan bank', 'csb bank', 'city union bank', 'dcb bank']

In [36]:
files = []
titles = []
dates = []
courts = []
cnr_nums = []
case_nums = []
petitioner = []
respondent = []
petitioner_advocates = []
respondent_advocates = []
judges = []
banks = []
citations = []
count = 0
for filename in list_of_filenames:
    count += 1
    if count > 5:
        break
    with open(os.path.join(os.path.join(head, 'sample_cases_10k', filename)), encoding="utf8") as f:
        soup = BeautifulSoup(f, 'html.parser')
        citation_banner = soup.find('div', 'doc_cite')
        if citation_banner:
            citation_banner.extract()
        text = soup.get_text().lower()
#         print(text)
#         soup.extract('div', 'doc_cite')
        files.append(filename)
        try:
            title, date = soup.title.string.split(' on ')
            date = datetime.strptime(date, "%d %B, %Y")
            titles.append(title)
            dates.append(date)
        except (AttributeError, ValueError):
            titles.append(np.nan)
            dates.append(np.nan)
        try:
            courts.append(soup.find("div", {"class": "docsource_main"}).text)
        except AttributeError:
            courts.append(np.nan)
        case_nums.append(ik.extract_case_num(text))
        petitioner.append(ik.extract_petitioner(text))
        respondent.append(ik.extract_respondent(text))
        pet_adv, resp_adv = ik.extract_petitioner_advocate(text)
        petitioner_advocates.append(pet_adv)
        respondent_advocates.append(resp_adv)
        cnr_nums.append(ik.extract_cnr(text))
        judges.append(ik.extract_judges(text, soup))
        banks.append(ik.extract_banks(text, list_of_banks))
        citations.append(ik.extract_citations(filename, str(soup)))
        
df = pd.DataFrame({'file': files, 'cnr_num': cnr_nums, 'title': titles, 'date': dates,
                   'court': courts, 'case_number': case_nums, 'petitioner': petitioner, 'respondent': respondent,
                   'petitioner_advocate': petitioner_advocates, 'respondent_advocate': respondent_advocates,
                   'judge': judges, 'banks': banks, 'citations': citations})
df

,file,cnr_num,title,date,court,case_number,petitioner,respondent,petitioner_advocate,respondent_advocate,judge,banks,citations
0,100002819.txt,NaN,M/S Supreme Brahmaputra (Jv) vs The State Of B...,2015-12-22,Patna High Court - Orders,civil writ jurisdiction case no.10232 of 2015,,,NaN,NaN,{mr. justice vikash jain},NaN,{152515230}
1,1000294.txt,NaN,Mirza Nausherwan Khan & Anr vs The Collector (...,1974-09-26,Supreme Court of India,NaN,NaN,NaN,NaN,NaN,"{Krishnaiyer, V.R.}",NaN,"{1251571, 7832, 780609}"
2,100092195.txt,NaN,Sri Nirakar Sahoo vs Sutar Chemical Private Lt...,2019-08-01,Supreme Court - Daily Orders,NaN,,,NaN,NaN,NaN,NaN,NaN
3,1000974.txt,NaN,Shiv Ji Sharma vs The State Of Bihar & Ors,2011-10-20,Patna High Court - Orders,NaN,,,NaN,NaN,"{sheema ali khan, j.}",NaN,{1965344}
4,100097838.txt,NaN,Divisional Manager National ... vs Satya Narai...,2011-01-13,Patna High Court - Orders,NaN,,,NaN,NaN,"{anjana prakash, j.}",NaN,NaN


In [13]:
df.citations[0]

{'1085825',
 '1251571',
 '1288162',
 '1861446',
 '780609',
 '7832',
 '859367',
 '979365'}